In [164]:
import csv
import pandas as pd
from requests import get
from json import dumps
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.lines import Line2D
import matplotlib.ticker as mtick
import numpy as np


In [126]:
_ONS = pd.read_csv("ONS_Nov18.csv")
ONS = _ONS.explode("Local authority areas")
_las = _ONS["Local authority areas"].to_list()
las = [x.replace("; ",";").split(";") for x in _las]
_ONS['LA'] = las
ONS = _ONS#.explode('LA')
ONS

,Map Number,Geography Code,Region,Local authority areas,Percentage testing positive,95% CI lower,95% CI upper,Interim IDs,LA
0,1,J06000101,North East,Northumberland,1.46%,0.84%,2.54%,36,[Northumberland]
1,2,J06000102,North East,North Tyneside,2.08%,1.14%,3.62%,77,[North Tyneside]
2,3,J06000103,North East,Newcastle upon Tyne,2.61%,1.53%,4.42%,76,[Newcastle upon Tyne]
3,4,J06000104,North East,South Tyneside; Sunderland; Gateshead,1.59%,0.99%,2.54%,78,"[South Tyneside, Sunderland, Gateshead]"
4,5,J06000105,North East,Darlington; County Durham,1.69%,1.04%,2.68%,2,"[Darlington, County Durham]"
...,...,...,...,...,...,...,...,...,...
111,112,J06000212,South West,East Devon; Exeter; Mid Devon; North Devon; Te...,0.68%,0.38%,1.23%,43,"[East Devon, Exeter, Mid Devon, North Devon, T..."
112,113,J06000213,South West,East Dorset; North Dorset; Purbeck; West Dorse...,0.72%,0.29%,1.88%,38,"[East Dorset, North Dorset, Purbeck, West Dors..."
113,114,J06000214,South West,Bournemouth; Poole; Christchurch,1.09%,0.54%,2.28%,37,"[Bournemouth, Poole, Christchurch]"
114,115,J06000215,South West,Cornwall; Isles of Scilly,0.49%,0.24%,0.95%,33,"[Cornwall, Isles of Scilly]"


In [3]:
dates = ["2020-11-22","2020-11-21","2020-11-20","2020-11-19","2020-11-18","2020-11-17","2020-11-16"]
PHE_df_list = []
for date in dates:
    filters = [
        f"areaType=ltla",
        f"date=%s" % date,
    ]

    structure = {
        "date": "date",
        "name": "areaName",
        "code": "areaCode",
        "newCasesBySpecimenDate":"newCasesBySpecimenDate",
    }

    api_params = {
        "filters": str.join(";", filters),
        "structure": dumps(structure, separators=(",", ":"))
    }

    response = get("https://api.coronavirus.data.gov.uk/v1/data", params=api_params, timeout=10)
    if response.status_code >= 400:
        raise RuntimeError(f'Request failed: { response.text }')
    response = response.json()
    PHE_df_list.append(pd.DataFrame.from_dict(response["data"]))
PHE = pd.concat(PHE_df_list)
PHE

,date,name,code,newCasesBySpecimenDate
0,2020-11-22,Aberdeen City,S12000033,13
1,2020-11-22,Aberdeenshire,S12000034,27
2,2020-11-22,Adur,E07000223,3
3,2020-11-22,Allerdale,E07000026,15
4,2020-11-22,Amber Valley,E07000032,39
...,...,...,...,...
375,2020-11-16,Wychavon,E07000238,35
376,2020-11-16,Wycombe,E07000007,48
377,2020-11-16,Wyre,E07000128,34
378,2020-11-16,Wyre Forest,E07000239,57


In [17]:
tiers = pd.read_csv("England_LAD_Covid_Tiers_Effective_02_12_2020.csv")
tiers

,LAD20CD,LAD20NM,GOV_NM,CTRY19CD,CTRY19NM,Tier
0,E06000001,Hartlepool,Hartlepool,E92000001,England,3
1,E06000002,Middlesbrough,Middlesbrough,E92000001,England,3
2,E06000003,Redcar and Cleveland,Redcar and Cleveland,E92000001,England,3
3,E06000004,Stockton-on-Tees,Stockton-on-Tees,E92000001,England,3
4,E06000005,Darlington,Darlington,E92000001,England,3
...,...,...,...,...,...,...
309,E09000029,Sutton,Sutton,E92000001,England,2
310,E09000030,Tower Hamlets,Tower Hamlets,E92000001,England,2
311,E09000031,Waltham Forest,Waltham Forest,E92000001,England,2
312,E09000032,Wandsworth,Wandsworth,E92000001,England,2


In [49]:
pillar_2_tests = pd.read_csv("P2_tests_processed_LTLA_w25.csv")
pillar_2_tests

,LTLA,LTLA Name,UTLA,UTLA Name,Region,Region Name,12/11/2020,13/11/2020,14/11/2020,15/11/2020,16/11/2020,17/11/2020,18/11/2020,Weekly total
0,NaN,ENGLAND,NaN,ENGLAND,NaN,ENGLAND,238675,244994,232467,185798,130835,162809,196261,1391839
1,E07000223,Adur,E10000032,West Sussex,E12000008,South East,348,147,192,234,70,204,177,1372
2,E07000026,Allerdale,E10000006,Cumbria,E12000002,North West,278,531,468,250,228,157,506,2418
3,E07000032,Amber Valley,E10000007,Derbyshire,E12000004,East Midlands,497,701,584,576,242,376,615,3591
4,E07000224,Arun,E10000032,West Sussex,E12000008,South East,833,1100,522,562,305,553,824,4699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,E07000229,Worthing,E10000032,West Sussex,E12000008,South East,415,447,503,469,110,394,304,2642
309,E07000238,Wychavon,E10000034,Worcestershire,E12000005,West Midlands,518,538,420,398,237,138,516,2765
310,E07000128,Wyre,E10000017,Lancashire,E12000002,North West,323,625,367,232,184,441,267,2439
311,E07000239,Wyre Forest,E10000034,Worcestershire,E12000005,West Midlands,361,788,333,577,210,161,449,2879


In [59]:
populations = pd.read_csv("population_stats.csv")
def get_pop_of(la_name):
    s = populations[populations.name==la_name].population.to_list()[0]
    return float(s.replace(",",""))
    
def get_PHE_pve_test(la):
    df  = PHE[PHE.name==la]
    counts = df.newCasesBySpecimenDate.to_list()
    return sum(counts)
        
def get_tier_of(la):
    return tiers[tiers.LAD20NM==la].Tier.values[0]

def get_pillar_2_test_of(la):
    return pillar_2_tests[pillar_2_tests["LTLA Name"]==la]["Weekly total"].values[0]

In [147]:
l = []
max_p = 0
for ltla in PHE.name.to_list():
    try:
        n = get_pillar_2_test_of(ltla)
        p = get_pop_of(ltla)
        l.append(n/p)
        max_p = max(max_p, p)
    except Exception as e:
        #print(ltla, str(e))
        pass
v = np.array(sorted(l))
min_val = np.percentile(v, 5)
max_val = np.percentile(v, 95)
print(min_val, max_val, max_p)

0.01528176298365465 0.03328521382531699 1141816.0


In [185]:
plt.clf()
figure(num=None, figsize=(32, 32), dpi=300, facecolor='w', edgecolor='k')

t_color = ["green", "yellow", "red"]
t_marker = ["v", "o", "^"]

def clamp(a,b,v):
    return max(min(v,b),a)

offset = 0.01
for idx, row in ONS.iterrows():
    x,p,n = 0,0,0
    t = []
    txt = row['Local authority areas']
    try:
        for ltla in row.LA:
            if ltla=="Liverpool":
                continue
            x += get_PHE_pve_test(ltla)
            p += get_pop_of(ltla)
            n += get_pillar_2_test_of(ltla)
            t.append(get_tier_of(ltla))
        x = x/p*100000
        y =float(row['Percentage testing positive'].strip("%"))
        c = clamp(0, 1, (n/p - min_val) / (max_val - min_val))
        #print(ltla, x, y, p, t, n, n/p, c)
        plt.scatter([x],[y], color=(c,0,1-c), marker=t_marker[t[0]-1], s=200*(p/max_p)**2)
        plt.annotate(txt, (x, y))
        offset *= -1
    except Exception as e:
        print("Exception", str(e), x, y, p, t)
        pass
    
    
custom_lines = [Line2D([0], [0], color=(1,0,0), lw=4),
                Line2D([0], [0], color=(0.5,0,0.5), lw=4),
                Line2D([0], [0], color=(0,0,1), lw=4),
                Line2D([0], [0], marker='v', color=(1,1,1), markerfacecolor=(0,0,0), markersize=12),
                Line2D([0], [0], marker='o', color=(1,1,1), markerfacecolor=(0,0,0), markersize=12),
                Line2D([0], [0], marker='^', color=(1,1,1), markerfacecolor=(0,0,0), markersize=12),
                Line2D([0], [0], marker='^', color=(1,1,1), markerfacecolor=(1,1,1), markersize=12),
               ]    

custom_lines_txt = ['Above avg tests per capita', 
                    'Average tests per capita', 
                    'Below avg tests per capita',
                    'Tier 1',
                    'Tier 2',
                    'Tier 3',
                    'Icon size = population size'
                   ]

plt.legend(custom_lines, custom_lines_txt, loc="upper left")
plt.ylabel("ONS modelled prevalence, % ")
plt.xlabel("Test&Trace positive tests per 100k population")

plt.savefig("normalization.png", bbox_inches='tight', pad_inches=0.1)
plt.show()
    
    

Exception list index out of range 0 0.46 0 []
Exception list index out of range 185 0.46 238597.0 [2, 2]
Exception index 0 is out of bounds for axis 0 with size 0 0 0.98 9721.0 []
Exception list index out of range 258 0.72 0 []
Exception list index out of range 476 0.65 407110.0 [2, 2, 2]
Exception list index out of range 0 0.68 0 []
Exception list index out of range 0 0.68 0 []
Exception index 0 is out of bounds for axis 0 with size 0 0 0.68 571802.0 [1]


<Figure size 432x288 with 0 Axes>